In [8]:
import pandas as pd
import numpy as np
import nibabel as nib
import seaborn as sns 
import os

import matplotlib.pyplot as plt
from neuromaps.images import relabel_gifti, load_data, load_gifti, annot_to_gifti
from neuromaps import parcellate
from neuromaps import transforms 
from neuromaps.stats import compare_images
from neuromaps.nulls import hungarian
from neuromaps.parcellate import Parcellater
from scipy.stats import pearsonr
from neuromaps.datasets import fetch_annotation

from matplotlib.cm import get_cmap
from surfplot import Plot
from neuromaps.datasets import fetch_fsaverage

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
## get the Turku and ENIGMA data

base_path = os.path.dirname(os.getcwd())

# load enigma & turku partial correlation data 
turku_data = pd.read_csv(os.path.join(base_path,'data', 'turku_partial_r.csv'))
turku_data = turku_data.rename(columns={'region_sample': 'region', 'partial_r_sample': 'partial_r'})

enigma_data = pd.read_csv(os.path.join(base_path,'data','ENIGMA_S32_partial_correlation_between_cortical_thickness_and_chlorpromazine_equivalents.csv'))
enigma_data = enigma_data[:-2]
enigma_data['region'] = turku_data['region']

In [5]:
# define path
path = os.path.dirname(os.getcwd()) + '/'

# load in different parcellation files
dk_fsaverage_10k = (path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-10k_hemi-L.label.gii.gz',
                     path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-10k_hemi-R.label.gii.gz')
dk_fsaverage_164k = (path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-164k_hemi-L.aparc-1.annot',
                     path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-164k_hemi-R.aparc-1.annot')
dk_mni = path + 'parcellations/atlas-desikankilliany_space-MNI_res-1mm.nii.gz'

# make sure label IDs are consecutive across hemispheres
dk_fsaverage_10k = relabel_gifti(dk_fsaverage_10k)
dk_fsaverage_164k = annot_to_gifti(dk_fsaverage_164k)  

# make the parcellaters for each space
parcellater_fs10k = Parcellater(dk_fsaverage_10k, 'fsaverage')
parcellater_fs164k = Parcellater(dk_fsaverage_164k, 'fsaverage')
parcellater_mni = Parcellater(dk_mni, 'MNI152')

# download the regions for MNI152, take indecies of surface rois  
rois = pd.read_csv(path+'parcellations/atlas-desikankilliany.csv')
rois = rois[(rois['structure'] == 'cortex')].index.to_numpy()

In [6]:
annotations = [('beliveau2017', 'dasb', 'fsaverage', '164k'),
 ('beliveau2017', 'cimbi36', 'fsaverage', '164k'),
 ('beliveau2017', 'sb207145', 'fsaverage', '164k')]

In [9]:
parcellated = dict([])

# go over each annotation and parcellate depending on the space 
for (src, desc, space, den) in annotations:
    annot = fetch_annotation(source=src, desc=desc, space=space, den=den)
    if space == 'MNI152':
        parcellater = parcellater_mni
    elif space == 'fsaverage' and den == '164k':
        parcellater = parcellater_fs164k
    elif space == 'fsLR' and den == '164k':
        space = 'fsaverage'
        annot = transforms.fslr_to_fsaverage(annot, target_density='164k')
        parcellater = parcellater_fs164k
    elif space == 'fsLR' and den != '164k':
        # unfortunately for fsLR-4k we are upsampling to fsaverage-10k to parcellate but it should be fine
        space = 'fsaverage'
        annot = transforms.fslr_to_fsaverage(annot, target_density='10k')
        parcellater = parcellater_fs10k

    parcellated[desc] = parcellater.fit_transform(annot, space=space, ignore_background_data=True)

    # if subcortex included remove 
    if parcellated[desc].shape == (1,83):
        parcellated[desc] = parcellated[desc][0][rois]

parcellated = pd.DataFrame(parcellated)

In [22]:
parcellated

,dasb,cimbi36,sb207145,region,enigma_partial_r,turku_partial_r
0,4.858744,58.675255,2.577081,lh_bankssts,-0.063,-0.141103
1,6.983108,44.467213,2.040007,lh_caudalanteriorcingulate,-0.009,0.006816
2,3.687662,55.912239,2.035033,lh_caudalmiddlefrontal,-0.102,-0.342748
3,6.035793,52.887188,1.761045,lh_cuneus,-0.031,-0.055788
4,9.050703,30.015448,2.234023,lh_entorhinal,-0.041,0.039894
...,...,...,...,...,...,...
63,4.214192,53.908180,2.408686,rh_supramarginal,-0.094,-0.284110
64,4.030276,58.710613,2.205722,rh_frontalpole,-0.043,-0.188452
65,8.621255,38.956345,2.119156,rh_temporalpole,-0.061,-0.141326
66,7.395446,53.104507,1.902578,rh_transversetemporal,-0.040,-0.172825


In [15]:
enigma_data['region']

0                    lh_bankssts
1     lh_caudalanteriorcingulate
2         lh_caudalmiddlefrontal
3                      lh_cuneus
4                  lh_entorhinal
                 ...            
63              rh_supramarginal
64                rh_frontalpole
65               rh_temporalpole
66         rh_transversetemporal
67                     rh_insula
Name: region, Length: 68, dtype: object

In [20]:
parcellated['region'] = enigma_data['region']
parcellated['enigma_partial_r'] = enigma_data['partial_r']
parcellated['turku_partial_r'] = turku_data['partial_r']

In [61]:
parcellated.columns = ['5HTT levels', '5HT2A levels', '5-HT4 levels','region', 'enigma_partial_r','turku_partial_r']
parcellated

,5HTT levels,5HT2A levels,5-HT4 levels,region,enigma_partial_r,turku_partial_r
0,4.858744,58.675255,2.577081,lh_bankssts,-0.063,-0.141103
1,6.983108,44.467213,2.040007,lh_caudalanteriorcingulate,-0.009,0.006816
2,3.687662,55.912239,2.035033,lh_caudalmiddlefrontal,-0.102,-0.342748
3,6.035793,52.887188,1.761045,lh_cuneus,-0.031,-0.055788
4,9.050703,30.015448,2.234023,lh_entorhinal,-0.041,0.039894
...,...,...,...,...,...,...
63,4.214192,53.908180,2.408686,rh_supramarginal,-0.094,-0.284110
64,4.030276,58.710613,2.205722,rh_frontalpole,-0.043,-0.188452
65,8.621255,38.956345,2.119156,rh_temporalpole,-0.061,-0.141326
66,7.395446,53.104507,1.902578,rh_transversetemporal,-0.040,-0.172825


In [67]:
def plot_correlations(x, y, xticks, yticks ):

    fig, ax = plt.subplots(figsize=(6, 6))
    plt.rcParams['axes.spines.right'] = False
    plt.rcParams['axes.spines.top'] = False
    sns.scatterplot(data=parcellated, x=x, y= y);
    sns.regplot(data=parcellated, x=x, y= y)
    ax.set_yticks(xticks) ; 
    ax.set_xticks(yticks) ; 
    ax.set_xlabel(x, size=16)
    ax.set_ylabel('Antipsychotic related cortical thinning', size=16)
    
    name = path+ '/figures/' + x.replace(' ','_') + y + '.jpg'
    plt.savefig(name, dpi=300,bbox_inches="tight")
    plt.close(fig) 


In [73]:
plot_correlations('5HTT levels', 'turku_partial_r',[-.35, -.15, 0],[4,6,8, 10])
plot_correlations('5HT2A levels', 'turku_partial_r',[-.35, -.15, 0], [30, 40, 50,60])
plot_correlations('5-HT4 levels', 'turku_partial_r',[-.35, -.15, 0], [1.5,2.0,2.5])

plot_correlations('5HTT levels', 'enigma_partial_r',[-.15, -.10, -0.05, 0],[4,6,8, 10])
plot_correlations('5HT2A levels', 'enigma_partial_r',[-.15, -.10, -0.05, 0], [30, 40, 50,60])
plot_correlations('5-HT4 levels', 'enigma_partial_r',[-.15, -.10, -0.05, 0], [1.5,2.0,2.5])